In [1]:
#importing required libraries

import pandas as pd
import requests

In [8]:
#acquiring the data, and converting it into a data frame. 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
temp = requests.get(url)

df_temp = pd.read_html(temp.content, header=0)[0]

#Ignore Boroughs where value is 'Not assigned'.
df = df_temp[df_temp.Borough != 'Not assigned']

df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
df.loc[df['Neighborhood'] == "Not assigned"]

,Postal Code,Borough,Neighborhood


#### No Row found where Neighborhood is Not Assigned.

In [16]:
#more than one neighborhood under the same postal code
df_tor = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(lambda x:', '.join(x))
df_tor = df_tor.reset_index()
df_tor


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [20]:
#printing size of dataframe
print ("The Dataframe df_tor has", df_tor.shape[0], 'rows.')

The Dataframe df_tor has 103 rows.
